In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using Test
using Plots
include("quadruped.jl")

 Activating environment at `~/Classes/OptimalControl/hw2/Project.toml`


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-15659412731956052003\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-8627523233044025172\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-15659412731956052003\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

run_tests (generic function with 1 method)

# Q1: Balancing the Quadruped with LQR (25 pts)
In this problem you'll stabilize the quadruped around the equilibrium point you found in the previous homework.

Note that we modified the model a little from the previous HW to make this problem easier. It now has once less degree of freedom (DOF), since we aren't allowing it to twist on the floor. Assuming we have "grippy" rubber feet, this isn't a bad assumption to make. This means it now has 28 states (14 positions, 14 velocities) and 12 controls.

In [2]:
# Load the quadruped model
model = UnitreeA1()
n,m = state_dim(model), control_dim(model)
dt = 0.01

# Use the equilibrium point we found in HW1
xeq = [0.30876598892362367, -0.260133570638461, -0.09358998727238592, 0.7078311865326717, -0.41038927424745475, -0.3598290230260733, 0.35837932714316556, 0.3579443729935737, -0.52325974352618, -0.524349602250509, 0.5321637004468419, 0.17483502352564967, 0.1745288580720117, -0.17241139464042135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ueq = [7.326746743908579, 12.492831761746691, -3.7276131797642345, -0.21210770886642585, 0.7228048164769122, 0.7061868479839222, -0.954088911642226, -0.8136046730618279, -0.5649940436402389, -0.273344715530589, -0.2382444936066465, -0.19338784324481662]

# Evaluate the continuous-time Jacobian
∇f = jacobian(model, xeq, ueq)
A = ∇f.A
B = ∇f.B

# Evaluate the discrete-time Jacobian
∇f = discrete_jacobian(RK4, model, xeq, ueq, dt)
A = ∇f.A
B = ∇f.B

LoadError: [91mUndefVarError: dt not defined[39m

## Part (a): Compute the optimal LQR gains using Riccati (8 pts)
As we saw in class, we can solve the LQR problem using a backward Riccati recursion, which also generates a locally-optimal feedback controller. Implement your own method to calculate a set of optimal feedback gains.

In [3]:
# TASK: Implement the following function (5 pts)
"""
    riccati(A,B,Q,R,Qf,N)

Use backward riccati recursion to solve the finite-horizon time-invariant LQR problem.
Returns vectors of the feedback gains `K` and cost-to-go matrices `P`, where `length(K) == N-1`,
`length(P) == N`, and `size(K[i]) == (m,n)` and `size(P[i]) == (n,n)`.

# Arguments:
* `A`: `(n,n)` discrete dynamics Jacobian wrt the state
* `B`: `(n,m)` discrete dynamics Jacobian wrt the control
* `Q`: `(n,n)` stage-wise cost matrix for states
* `R`: `(m,m)` stage-wise cost matrix for controls
* `Qf`: `(n,n)` cost matrix for terminal state
* `N`: Integer number of time steps (horizon length).
"""
function riccati(A,B,Q,R,Qf,N)
    # initialize the output
    n,m = size(B)
    P = [zeros(n,n) for k = 1:N]
    K = [zeros(m,n) for k = 1:N-1]
    
    # TODO: implement the Riccati recursion
    
    # return the feedback gains and ctg matrices
    return K,P
end

riccati

In [4]:
# TASK: Compute the optimal gains (3 pts)
#       Store the result in a matrix K::Vector{Matrix{Float64}}, where size(K[1]) = (12,28)
#       Use the cost matrices provided below

# Some parameters
dt = 0.01
tf = 2.0
times = range(0,tf,step=dt)
N = length(times)

# Define the LQR cost
Q = Diagonal([fill(130.,n÷2); fill(130.,n÷2)])
R = Diagonal(fill(10.0, m))
Qf = copy(Q)

# TODO: solve for the feedback gains
A = zeros(n,n)
B = zeros(n,m)
K,P = riccati(A,B,Q,R,Qf,N);

## Part (b): Implement the LQR Controller (3 pts)
Now that we've computed our gains, let's implement a controller to use on our system!

In [ ]:
# TASK: implement the following methods
#       get_control
"""
    LQRController

Type for evaluting a time-invariant LQR control policy. If the `infinite_horizon`
field is true, it will only use a single gain, `K[1]`.
"""
struct LQRController
    K::Vector{Matrix{Float64}}   # feedback gains ((m,n),N-1)
    times::Vector{Float64}       # times          (N,)
    xeq::Vector{Float64}         # equilibrium states
    ueq::Vector{Float64}         # equilibrium controls
    infinite_horizon::Bool       # use infinite horizon control
end
function LQRController(K,xeq,ueq,tf, ih=false)
    LQRController(Matrix.(K), collect(range(0,tf,length=length(K)+1)), xeq, ueq, ih)
end

"""
    get_k(ctrl, t)

Get the time index corresponding to time `t`. 
Useful for implementing zero-order hold control.
Uses binary search to find the time index.
"""
get_k(controller::LQRController, t) = searchsortedlast(controller.times, t)

"""
    get_control(ctrl::LQRController, x, t)

Evaluate the LQRController feedback policy at state `x` and time `t`, returning the control 
to be executed by the system.
"""
function get_control(controller::LQRController, x, t)
    # TODO: Finish the function to calculate the control at time t
    return zero(controller.ueq)
end

In [ ]:
# Test the controller
ctrl = LQRController(K,xeq,ueq,tf)
utest = get_control(ctrl, xeq, 0.0) - ueq
@test norm(utest) ≈ 0

## Part (c): Compute the infinite-horizon gain (6 pts)
In the previous question, we found a set of time-varying gains to stabilize our system. However, let's look at the behavior of our gains over time.

In [ ]:
using Plots
Kmat = hcat(vec.(K)...);
plot(times[1:end-1], Kmat', legend=:none, xlabel="time (s)", ylabel="feedback gains")

It looks like they're starting to converge to some steady-state initial gain. Let's increase the length of our horizon and see what happens.

In [ ]:
# TODO: Extend the horizon by at least 2x and save the new gains and cost-to-go in Kinf, Pinf (3 pts)
Kinf,Pinf = deepcopy(K), deepcopy(P)

# Plot the result
Kmat = hcat(vec.(Kinf)...);
plot(Kmat', legend=:none, xlabel="time (s)", ylabel="feedback gains")

You should be able to see that they're definitely converging to some steady-state initial value. This value is usually referred to as the "infinite horizon" gain, and is very useful in practice. It's common practice to use the associated cost-to-go matrix as the $Q_f$ weighting matrix in finite-horizon problems (we'll do this in Q3).

**TASK**: Tweak your controller code above so that if the `infinite_horizon` flag is true, it only uses the gains at the first time step. Then generate a new controller, `ctrl_inf` that only uses the infinite gain.

With both the finite and infinite-horizon controllers defined, let's see how they do on the system.

In [ ]:
# TODO: create the infinite-horizon controller (3 pts)
ctrl_inf = ctrl

## Part (d): Stability Analysis (3 pts)
Before we simulate, it's always a good idea to check the stability of our system. Evaluate the stability using the infinite-horizon gain and

In [ ]:
# TASK: compute the stability of our original uncontrolled system
#       i.e. norm of largest eigenvalue
stability0 = NaN

# TASK: compute the stability of our new, controlled system
stability = NaN

@test stability0 > 1
@test stability < 1

## Part (e): Simulate the Quadruped (5 pts)
Let's now put it all together and stabilize the quadruped.

In [ ]:
# TASK: complete the simulator code (5 pts)
"""
    simulate(model, x0, controller; dt, tf, mvis)

Simulate the quadruped, starting from initial state `x0` and using `controller` to stabilize the system.

# Keyword Arguments
* `tf`: total simulation time
* `dt`: simulation time step
* `mvis`: if passed in, use the `MechanismVisualizer` to visualize the simulation while it's running
"""
function simulate(model::UnitreeA1, x0, controller; dt=0.05, tf=1.0, mvis=nothing)
    # some initialization
    time = range(0, tf, step=dt)
    n,m = state_dim(model), control_dim(model) 
    N = Int(round(tf/dt)) + 1
    X = [@SVector zeros(n) for k = 1:N] 
    U = [@SVector zeros(m) for k = 1:N-1] 
    
    # set the initial state
    X[1] = x0

    for k = 1:length(time) - 1
        # TODO: simulate the system with feedback

        # visualization code
        if !isnothing(mvis)
            set_configuration!(mvis, X[k+1][1:14])
            sleep(dt)
        end
    end
    return X,U,time
end

In [ ]:
mvis = initialize_visualizer(model)
render(mvis)

In [ ]:
# Perturb the initial condition
x_init = copy(xeq)
x_init[18] -= 0.1
x_init[19] -= 0.05

In [ ]:
# Finite Horizon
X,U,time = simulate(model, x_init, ctrl, dt=dt, tf=tf);
visualize!(mvis, model, time[end], X)  # send the trajectory to the visualizer

In [ ]:
# Infinite horizon
Xinf,Uinf,time = simulate(model, x_init, ctrl_inf, dt=dt, tf=tf);
visualize!(mvis, model, time[end], Xinf)  # send the trajectory to the visualizer

Let's take a look some plots.

In [ ]:
# Plot the response
c = [:red :green :blue]
dX = [x - xeq for x in X]
plot(times,dX,inds=3:5, legend=:none, xlabel="time (s)", ylabel="joint positions (rad)", c=c)
dX2 = [x - xeq for x in Xinf]
plot!(times, dX2, inds=3:5, legend=:none, colors=1:12, ls=:dash, c=c)

In [ ]:
dU = [u - ueq for u in U]
plot(times[1:end-1], dU, inds=1:3, xlabel="time (s)", ylabel="joint torques", c=c)
dU2 = [u - ueq for u in Uinf]
plot!(times[1:end-1], dU2, inds=1:3, ls=:dash, c=c)

## Tests

In [ ]:
run_tests();